# Plume Tool

## What is a plume? 
A plume is characterized by rising buoyant fluids where the buoyancy forces arise from intense localized sources [[1]](https://www.sciencedirect.com/science/article/abs/pii/B978012386660850009X). Thermal plumes (super-heated smoke plumes) are large, tall volumes of hot material that form over large, intense wildfires. Sometimes VIIRS instruments detect the surface fire along with the plume and incorrectly categorize the plume as an active fire. Hot plumes carry a signature which is easily confused with that of the “real” fire perimeter. 

This happens when ([as reported here](https://www.earthdata.nasa.gov/faq/firms-faq#ed-false-detections)): 

1. Parallax effect causes tall/superheated plume detection pixel(s) to be displaced laterally when projected onto the surface
2. Tall, superheated plumes carrying large volumes of hot material into the air are formed over large and intense wildfires, and VIIRS detects the surface fire along with part of the plume and categorizes them as active fires

[FIRMS](https://www.earthdata.nasa.gov/faq/firms-faq#ed-false-detections) also discusses when false positives from superheated plumes are most likely to occur: 

1. Nighttime: This is the period during which the VIIRS active fire product is particularly responsive to heat sources thereby favoring plume detection

2. Very large wildfires undergoing explosive growth and acompanied by rapid/vertically elongated plume development. Enough hot material must entrain the plume creating a distinguishable thermal signal (i.e., one that significantly exceeds the fire-free surface background)

3. High scan angle: This is what will ultimately produce the detections extending beyond the actual fire perimeter

They recommend using observations from other sources for these cases. 

## Brainstorming Solutions

### Identifying hot spots which are outside of known fire perimeters

Questions from KZ: 
- How are the final perimeters created? Will they be affected by the false detections? If they use satellite imagery and the hotspot detection algorithms, can the final fire perimeters be used at all to identify false positives?
- NBAC is commonly used in Canada and MTBS in the states as final fire perimeters, is there any error in this assessment?

1. Identify hot spots which are outside of final fire perimeters reported by an agency (such as NBAC, NFDB, MTBS)

2. Remove known cases for false positives (sunglint, water bodies, persistent heat sources which correspond to false positives)

3. Explore day of burning perimeters for the remaining false positives, which can be created using similar methods as done in [Balik et al. 2024](https://www.frontiersin.org/journals/forests-and-global-change/articles/10.3389/ffgc.2024.1355361/full), [Parks et al. 2014](https://www.publish.csiro.au/wf/WF13138), and [Barber et al. 2024](https://www.nature.com/articles/s41597-024-03436-4). The day of burning perimeters are usually interpolated from VIIRS/MODIS hotspots. These will be affected by the false detections.

### Other calculated/measured variables 

1. FRP has been shown to be a good indication of false positive detections [[2]](https://www.mdpi.com/2220-9964/11/12/601). Maybe FRP can be used to lower confidence of hot spot detections if there is some statistical significance between:
    - The FRP of points within the perimeter and outside of the perimeter
    - The FRP of fires corresponding to hot superheated smoke plumes compared to fires with no superheated smoke plumes (How hot are plumes compared to regular fires?)

2. 100 m wind speeds might be able to be used to identify "blown" plumes, along with the height of the plume 
    - Can we encorporate wind speed to see if the plume is being pushed in the wind direction

4. How hot are the plumes?
    - Can we use bands from satellite imagery to identify heat within the plumes to set a threshold for detecting them?
    - What is the temperature that must be reached for a VIIRS detection? Can we get a list of hot smoke plumes and find more characteristics about them this way?
  
### Other Datasets of Smoke Plumes

1. [MISR Plume Height Project 2](https://misr.jpl.nasa.gov/get-data/misr-plume-height-project-2/)
    - Publicly available database of wildfire smoke plume heights
    - [MERLIN](https://l0dup05.larc.nasa.gov/merlin/merlin)

3. [USTC_SmokeRS](https://onedrive.live.com/?authkey=%21AFYQkl1tP%2DQh3Ek&id=2B888FC2F8F47809%21857&cid=2B888FC2F8F47809)
    - The dataset is collected based on Moderate Resolution Imaging Spectroradiometer (MODIS) data, consisting of 6225 RGB images from six classes including Cloud, Dust, Haze, Land, Seaside, and Smoke

## This Notebook

[This blog](https://landweb.modaps.eosdis.nasa.gov/displayissue?id=339) has listed some examples of known plumes within the United States. A metadata file was created by referencing these plumes and is used as input to this notebook. This notebook uses GEE and FIRMS to explore hot spots (VIIRS SNPP, VIIRS NOAA20, MODIS, GOES) and corresponding satellite imagery (Landsat, VIIRS, GOES) to aid in the identification of false fire detections caused by the presence of superheated plumes. These false detections can be located outside of the reported fire perimeter of a large wildfire. 

You can export the clipped tiff files from Google Earth Engine using this notebook as well, but we found exploring interactively using geemap made more sense in this instance.

In [13]:
import io
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import time
from glob import glob
import rasterio as rio
from googleapiclient.discovery import build
from google.oauth2 import service_account
from googleapiclient.http import MediaIoBaseDownload
import numpy as np
from osgeo import gdal
gdal.SetConfigOption('SHAPE_RESTORE_SHX', 'YES')

import ee
import geemap
ee.Authenticate()

True

## Functions

In [1]:
def get_info(r):
    # Filled in by sheet

    id = str(r['id'])

    # Coords of hotspots to pull, will also be used as bounding box for satellite imagery in GEE
    coords = str(r['coord-0']) + ',' + str(r['coord-1']) + ',' + str(r['coord-2']) + ',' + str(r['coord-3'])

    # Number of days surrounding date to pull hotspots
    VIIRS_no_days = int(r['viirs-no-days'])

    # Date to pull hotspots
    VIIRS_date = str(r['viirs-date']).split(' ')[0]

    # Start and end dates for filtering in google
    start_date = str(r['gee-start-date']).split(' ')[0]
    end_date = str(r['gee-end-date']).split(' ')[0]

    return id, coords, VIIRS_no_days, VIIRS_date, start_date, end_date

In [2]:
def obtain_viirs_hotspots(FIRMS_map_key, coords, VIIRS_no_days, VIIRS_date):
    """

    Returns VIIRS hotspots from FIRMS for specified date, coordinates, and date range in the form of a
    geodataframe.

    :param FIRMS_map_key:
    :param VIIRS_coords:
    :param VIIRS_no_days:
    :param VIIRS_date:
    :return:
    """

    MAP_KEY = FIRMS_map_key
    url = 'https://firms.modaps.eosdis.nasa.gov/mapserver/mapkey_status/?MAP_KEY=' + MAP_KEY
    try:
        df = pd.read_json(url, typ='series')
    except:
        # possible error, wrong MAP_KEY value, check for extra quotes, missing letters
        print("There is an issue with the query. \nTry in your browser: %s" % url)

    # VIIRS_S-NPP
    area_url_SNPP = ('https://firms.modaps.eosdis.nasa.gov/api/area/csv/' + MAP_KEY + '/VIIRS_SNPP_SP/' +
                     str(coords) + '/' + str(VIIRS_no_days) + '/' + str(VIIRS_date))
    df_SNPP = pd.read_csv(area_url_SNPP)

    gdf_SNPP = gpd.GeoDataFrame(
        df_SNPP, geometry=gpd.points_from_xy(df_SNPP.longitude, df_SNPP.latitude), crs="EPSG:4326"
    )

    return gdf_SNPP

In [64]:
def apply_scale_and_offset(image):

    # Band aliases.
    BLUE = 'CMI_C01'
    RED = 'CMI_C02'
    VEGGIE = 'CMI_C03'
    GREEN = 'GREEN'
    
    # Number of bands in the EE asset, 0-based.
    NUM_BANDS = 33
    
    # Skipping the interleaved DQF bands.
    BLUE_BAND_INDEX = (1 - 1) * 2
    RED_BAND_INDEX = (2 - 1) * 2
    VEGGIE_BAND_INDEX = (3 - 1) * 2
    GREEN_BAND_INDEX = NUM_BANDS - 1
    
    # Visualization range for GOES RGB.
    GOES_MIN = 0.0
    GOES_MAX = 0.7  # Alternatively 1.0 or 1.3.
    GAMMA = 1.3
    bands = [None] * NUM_BANDS  # Initialize with None to ensure correct length.
    
    for i in range(1, 17):
        band_name = f'CMI_C{str(100 + i)[-2:]}'
        offset = ee.Number(image.get(f'{band_name}_offset'))
        scale = ee.Number(image.get(f'{band_name}_scale'))
        bands[(i - 1) * 2] = image.select(band_name).multiply(scale).add(offset)

        dqf_name = f'DQF_C{str(100 + i)[-2:]}'
        bands[(i - 1) * 2 + 1] = image.select(dqf_name)

    # Green = 0.45 * Red + 0.10 * NIR + 0.45 * Blue
    green1 = bands[RED_BAND_INDEX].multiply(0.45)
    green2 = bands[VEGGIE_BAND_INDEX].multiply(0.10)
    green3 = bands[BLUE_BAND_INDEX].multiply(0.45)
    green = green1.add(green2).add(green3)
    bands[GREEN_BAND_INDEX] = green.rename(GREEN)

    return ee.Image(ee.Image(bands).copyProperties(image, image.propertyNames()))

In [62]:
def download_and_plot_gee_data(id, coords, start_date, end_date, VIIRS_date, google_drive_folder, landsat_flag, export_flag):
    """

    :param id:
    :param coords:
    :param VIIRS_date:
    :param google_drive_folder:
    :param landsat_flag:
    :return:
    """

    Map = geemap.Map()
    
    roi = ee.Geometry.Rectangle([float(coords.split(',')[0]), float(coords.split(',')[1]),
                                 float(coords.split(',')[2]), float(coords.split(',')[3])])

    date_of_interest = ee.Date(VIIRS_date)

    ## VIIRS TRUE COLOUR
    viirs = ee.ImageCollection("NASA/VIIRS/002/VNP09GA").filterDate(start_date, end_date).filterBounds(roi)

    rgb_viirs_tc = viirs.select(['M5', 'M4', 'M3'])

    # Subtract the time of each image in collection from date of interest
    rgb_viirs_tc_sort = rgb_viirs_tc.map(lambda image: image.set(
        'dateDist',
        ee.Number(image.get('system:time_start')).subtract(date_of_interest.millis()).abs()
    ))

    # sort in ascending order by dateDist (so top image will correspond to date of interest)
    viirs_ic_rc_sorted = rgb_viirs_tc_sort.sort('dateDist')

    # grab the first image from the sorted image collection
    img_viirs_tc = viirs_ic_rc_sorted.first()

    # clip the image to the roi
    clipped_viirs_tc = img_viirs_tc.clip(roi)

    rgb_vis_viirs_tc = {'min': 0.0, 'max': 0.3}

    Map.addLayer(clipped_viirs_tc, rgb_vis_viirs_tc, 'Clipped-' + str(id) + '-viirs-tc', shown=0)
    Map.addLayer(img_viirs_tc, rgb_vis_viirs_tc, 'Full-' + str(id) + '-viirs-tc', shown=0)

    if export_flag == 1:
    # export using visualization parameters suggested by GEE
        export_image_viirs_tc = clipped_viirs_tc.select('M.*').visualize(min=0, max=0.4)
        export_gee_data(id, export_image_viirs_tc, roi, 'v1', google_drive_folder)

    ## VIIRS FALSE COLOUR
    rgb_viirs_fc = viirs.select(['I3', 'I2', 'I1'])

    # Subtract the time of each image in collection from date of interest
    rgb_viirs_fc_sort = rgb_viirs_fc.map(lambda image: image.set(
        'dateDist',
        ee.Number(image.get('system:time_start')).subtract(date_of_interest.millis()).abs()
    ))

    # sort in ascending order by dateDist (so top image will correspond to date of interest)
    viirs_ic_fc_sorted = rgb_viirs_fc_sort.sort('dateDist')

    # grab the first image from the sorted image collection
    img_viirs_fc = viirs_ic_fc_sorted.first()

    # clip the image to the roi
    clipped_viirs_fc = img_viirs_fc.clip(roi)

    Map.addLayer(clipped_viirs_fc, rgb_vis_viirs_tc, 'Clipped-' + str(id) + '-viirs-fc', shown=0)
    Map.addLayer(img_viirs_fc, rgb_vis_viirs_tc, 'Full-' + str(id) + '-viirs-fc', shown=0)

    if export_flag == 1:
        # export using visualization parameters suggested by GEE
        export_image_viirs_fc = clipped_viirs_fc.select('I.*').visualize(min=0, max=0.4)
        export_gee_data(id, export_image_viirs_fc, roi, 'v2', google_drive_folder)


    ## landsat

    if landsat_flag == 1:

        landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA").filterDate(start_date, end_date).filterBounds(roi)

        true_color_432 = landsat.select(['B4', 'B3', 'B2'])
        true_color_432_vis = {'min': 0.0, 'max': 0.4}

        # Because landsat is very slow, we want to use as much data as possible
        # So we'll take the median value instead of sorting by date like we did for daily viirs
        image_landsat = true_color_432.median()

        clipped_landsat = image_landsat.clip(roi)

        Map.addLayer(clipped_landsat, true_color_432_vis, 'Clipped-' + str(id) + '-landsat', shown=0)
        Map.addLayer(image_landsat, true_color_432_vis, 'Full-' + str(id) + '-landsat', shown=0)

        if export_flag == 1:

            export_image_landsat = clipped_landsat.select('B.*').visualize(min=0, max=0.4)
            export_gee_data(id, export_image_landsat, roi, 'l', google_drive_folder)

    # GOES
    goes = ee.ImageCollection("NOAA/GOES/16/MCMIPC").filterDate(start_date, end_date).filterBounds(roi)

    goes_sort = goes.map(lambda image: image.set(
            'dateDist',
            ee.Number(image.get('system:time_start')).subtract(date_of_interest.millis()).abs()))

    # sort in ascending order by dateDist (so top image will correspond to date of interest)
    goes_sorted = goes_sort.sort('dateDist')

    # grab the first image from the sorted image collection
    goes_img = goes_sorted.first()

    # clip the image to the roi
    goes_img_clipped = goes_img.clip(roi)

    goes_img_tc = apply_scale_and_offset(ee.Image(goes_img_clipped))

    goes_rgb_viz = {'bands': [RED, GREEN, BLUE], 'min': GOES_MIN, 'max': GOES_MAX, 'gamma': GAMMA}

    Map.addLayer(goes_img_tc, goes_rgb_viz, 'goes')
    
    Map.centerObject(roi, 9)
    return Map

In [4]:
def export_gee_data(id, exportImage, roi, flag, google_drive_folder):
    """
    Export data from google earth engine to google drive
    :param exportImage:
    :param roi:
    :param flag:
    :param google_drive_folder:
    :return:
    """

    if flag=='v1':
        # Define the export parameters
        export_task = ee.batch.Export.image.toDrive(
            image=exportImage,
            description='VIIRS_RGB_True_Export',
            folder=google_drive_folder,  # Change this to your preferred folder in Google Drive
            fileNamePrefix= id + '-viirs_true_rgb',
            region=roi,  # Define the region to export
            scale=500,  # Scale in meters
            crs='EPSG:4326',  # Coordinate reference system
            maxPixels=1e13  # Maximum number of pixels to export
        )

    elif flag=='v2':
        # Define the export parameters
        export_task = ee.batch.Export.image.toDrive(
            image=exportImage,
            description='VIIRS_RGB_False_Export',
            folder=google_drive_folder,  # Change this to your preferred folder in Google Drive
            fileNamePrefix= id + '-viirs_false_rgb',
            region=roi,  # Define the region to export
            scale=500,  # Scale in meters
            crs='EPSG:4326',  # Coordinate reference system
            maxPixels=1e13  # Maximum number of pixels to export
        )

    elif flag=='l':
        # Define the export parameters
        export_task = ee.batch.Export.image.toDrive(
            image=exportImage,
            description='Landsat',
            folder=google_drive_folder,  # Change this to your preferred folder in Google Drive
            fileNamePrefix= id + '-landsat-truecolour',
            region=roi,  # Define the region to export
            scale=30,  # Scale in meters
            crs='EPSG:4326',  # Coordinate reference system
            maxPixels=1e13  # Maximum number of pixels to export
        )

    # Start the export task
    export_task.start()

    while export_task.active():
        print('Polling for task (id: {}).'.format(export_task.id))
        time.sleep(10)

    print('Task completed with status: ', export_task.status())


In [5]:
def drive_download_data():
    """
    This directly downloads files from my Google Drive in the GEE_Exports folder
    :return:
    """

    creds = service_account.Credentials.from_service_account_file(
        creds_file,
        scopes=['https://www.googleapis.com/auth/drive']
    )

    drive_service = build('drive', 'v3', credentials=creds)

    # First, get the folder ID by querying by mimeType and name
    folderId = drive_service.files().list(q="mimeType = 'application/vnd.google-apps.folder' and name = 'GEE_Exports'",
                                  pageSize=10, fields="nextPageToken, files(id, name)").execute()
    # this gives us a list of all folders with that name
    folderIdResult = folderId.get('files', [])
    # however, we know there is only 1 folder with that name, so we just get the id of the 1st item in the list
    id = folderIdResult[0].get('id')

    # Now, using the folder ID gotten above, we get all the files from
    # that particular folder
    results = drive_service.files().list(q="'" + id + "' in parents", pageSize=10,
                                 fields="nextPageToken, files(id, name)").execute()
    items = results.get('files', [])

    for item in items:

        # NOTE: Had to give permission to certificate email to see the folder: google-drive@karlzam.iam.gserviceaccount.com
        # Replace this with getting all the files within the GEE folder
        path = download_path
        file_path = path + '\\' + str(item['name'])

        request = drive_service.files().get_media(fileId=item['id'])

        fh = io.FileIO(file_path, mode='wb')
        downloader = MediaIoBaseDownload(fh, request)
        done = False

        while not done:
            status, done = downloader.next_chunk()


In [6]:
def plot_plume(id, hotspots, tif_folder, plot_folder):
    """

    :param hotspots:
    :param tif_folder:
    :param plot_folder:
    :return:
    """

    files = glob(tif_folder + '\\' + '*.tif')

    sub_files = []
    for ii in range(0, len(files)):
        if id in files[ii]:
            sub_files.append(files[ii])

    for tif_file in sub_files:

        if 'viirs_true' in tif_file:

            source = 'viirs-true'
            hotspot_plot = True
            plot_rgb(id, tif_file, hotspots, plot_folder, source, hotspot_plot)

        elif 'viirs_false' in tif_file:
            source = 'viirs_false'
            hotspot_plot = False
            plot_rgb(id, tif_file, hotspots, plot_folder, source, hotspot_plot)

        if 'landsat' in tif_file:

            source = 'landsat'
            hotspot_plot = True
            plot_rgb(id, tif_file, hotspots, plot_folder, source, hotspot_plot)

In [7]:
def plot_rgb(id, tif_file, hotspots, plot_folder, source, hotspot_plot):

    with rio.open(tif_file) as src:
        # Read the image data
        # img_data = src.read(1)
        b1 = src.read(1)
        b2 = src.read(2)
        b3 = src.read(3)

        rgb = np.dstack((b1, b2, b3))

        bounds = src.bounds

        # Plotting
        plt.figure(figsize=(10, 10))
        plt.imshow(rgb, extent=[bounds.left, bounds.right, bounds.bottom, bounds.top])

        if hotspot_plot:
            plt.scatter(hotspots['longitude'], hotspots['latitude'], s=1, alpha=0.3, color='red',
                        label='VIIRS-SNPP Hot Spots')
        plt.title(tif_file.split('\\')[-1])
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        plt.legend()
        plt.savefig(plot_folder + '\\' + str (id) + '-' + str(source) +  '.png')

## Main

In [32]:
plume_excel_sheet = pd.read_excel(r'C:\Users\kzammit\Documents\Plumes\plume-metadata.xlsx')

plot_path = r'C:\Users\kzammit\Documents\Plumes\plots'

### FIRMS Related Variables ###

# Map key
FIRMS_map_key = 'e865c77bb60984ab516517cd4cdadea0'

### GEE Related Variables ###

gee_proj = 'karlzam'

# DO NOT CHANGE THIS - hardcoded later on
# For other users: you need to share this folder with your credential for google API for google cloud
google_drive_folder = 'GEE_Exports'

# Where satellite data will be exported to
download_path = r'C:\Users\kzammit\Documents\Plumes\downloaded-tifs'

# Before running this script:
# Create a service key for your google cloud project and download the .json to your local machine
creds_file = r'C:\Users\kzammit\Documents\google-drive-API-key\karlzam-d3258a83d6cb.json'

In [65]:
if __name__ == "__main__":

    #ee.Initialize(project=gee_proj)
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

    Maps = []
    hotspots = []

    for ii in range(0, len(plume_excel_sheet)):

        #print('reading excel sheet')
        id, coords, VIIRS_no_days, VIIRS_date, start_date, end_date = get_info(plume_excel_sheet.iloc[ii])

        #print('obtaining VIIRS hotspots')
        viirs_hotspots = obtain_viirs_hotspots(FIRMS_map_key, coords, VIIRS_no_days, VIIRS_date)

        hotspots.append(viirs_hotspots)

        #print('accessing and downloading gee imagery to drive')
        # WARNING: The Landsat scale is currently set to export at 50m, and exporting the .tif takes quite a while!
        landsat_flag = 1
        # if you want to export your data as well as plotting interactively
        export_flag = 0
        Map = download_and_plot_gee_data(id, coords, start_date, end_date, VIIRS_date, google_drive_folder, landsat_flag, export_flag)

        #print('downloading data from drive to local path')
        #drive_download_data()

        #plot_plume(id, viirs_hotspots, download_path, plot_path)
        print('Completed fire: ' + str(id))

        Maps.append(Map)

Completed fire: Carr
Completed fire: Holy
Completed fire: Museum
Completed fire: Kincade
Completed fire: Mineral


In [66]:
# Change this to look at the specific fire (ordered like the excel sheet)
# This converts the hotspots into a geo interface so I can plot overtop of the map
geojson = hotspots[0].__geo_interface__
Maps[0].add_geojson(geojson, layer_name='Hotspots', style={'color': 'blue', 'weight': 2})
Maps[0]

Map(center=[40.502248236778364, -121.9000000000002], controls=(WidgetControl(options=['position', 'transparent…